In [1]:
# !pip install tensorflow

In [2]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

## Loading imgs

In [3]:
# !pip install opencv-python

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [6]:
# data folders
train_data_folder = 'RawData_img/train'
val_data_folder = 'RawData_img/val'
test_data_folder = 'RawData_img/test'

In [7]:
# image dimensions
image_height = 128
image_width = 128
batch_size = 32

In [8]:
# Create an instance of ImageDataGenerator with rescaling
datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# Create generators for training, validation, and testing data
train_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    val_data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for the validation set
)

test_generator = datagen.flow_from_directory(
    test_data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False for the test set
)

Found 15892 images belonging to 4 classes.
Found 1986 images belonging to 4 classes.
Found 1989 images belonging to 4 classes.


In [6]:
print(train_generator.class_indices)

{'ambos': 0, 'azz': 1, 'esp': 2, 'na': 3}


# CNN model

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_height, image_width, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 4 classes categories

In [10]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

C:\Users\natas\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
# Train the model using the training set and validate on the validation set
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/10
497/497 [==============================] - 487s 975ms/step - loss: 0.4041 - accuracy: 0.8229 - val_loss: 0.0746 - val_accuracy: 0.9768
Epoch 2/10
497/497 [==============================] - 403s 810ms/step - loss: 0.0707 - accuracy: 0.9749 - val_loss: 0.0324 - val_accuracy: 0.9874
Epoch 3/10
497/497 [==============================] - 408s 820ms/step - loss: 0.0459 - accuracy: 0.9820 - val_loss: 0.0451 - val_accuracy: 0.9894
Epoch 4/10
497/497 [==============================] - 404s 812ms/step - loss: 0.0408 - accuracy: 0.9838 - val_loss: 0.0360 - val_accuracy: 0.9879
Epoch 5/10
497/497 [==============================] - 413s 831ms/step - loss: 0.0359 - accuracy: 0.9862 - val_loss: 0.0338 - val_accuracy: 0.9899
Epoch 6/10
497/497 [==============================] - 439s 882ms/step - loss: 0.0292 - accuracy: 0.9879 - val_loss: 0.0304 - val_accuracy: 0.9914
Epoch 7/10
497/497 [==============================] - 446s 896ms/step - loss: 0.0253 - accuracy: 0.9882 - val_loss: 0.0341 -

In [12]:
# Save model
model.save('Models/slid_cnn_v1_e10.h5')

In [13]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

63/63 [==============================] - 44s 699ms/step - loss: 0.0231 - accuracy: 0.9930
Test Accuracy: 99.30%


In [14]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [15]:
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
conf_matrix = confusion_matrix(true_labels, predicted_labels)

63/63 [==============================] - 15s 231ms/step


In [16]:
# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[ 25  10   1   0]
 [  0 815   0   0]
 [  1   1 773   1]
 [  0   0   0 362]]


In [17]:
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.69      0.81        36
           1       0.99      1.00      0.99       815
           2       1.00      1.00      1.00       776
           3       1.00      1.00      1.00       362

    accuracy                           0.99      1989
   macro avg       0.99      0.92      0.95      1989
weighted avg       0.99      0.99      0.99      1989



In [18]:
## load model 
# model = load_model('Models/slid_cnn_v1.h5')

## Logs

| Model | Version | Epochs | Train Loss | Train Accuracy | Val Accuracy | Test Accuracy |
|-------|---------|--------|------------|----------------|--------------|---------------|
| CNN   | v1      |   1    |            |                |              |   95.78%      |
| CNN   | v1      |  5     |            |                |              |   98.99%      |



In [ ]:
################################################################

In [1]:
from tensorflow.keras.models import load_model
model = load_model('Models/slid_cnn_v1_e10.h5')

C:\Users\natas\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [10]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

63/63 [==============================] - 95s 1s/step - loss: 0.0231 - accuracy: 0.9930
Test Accuracy: 99.30%


In [11]:
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
conf_matrix = confusion_matrix(true_labels, predicted_labels)

63/63 [==============================] - 15s 239ms/step


In [12]:
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.69      0.81        36
           1       0.99      1.00      0.99       815
           2       1.00      1.00      1.00       776
           3       1.00      1.00      1.00       362

    accuracy                           0.99      1989
   macro avg       0.99      0.92      0.95      1989
weighted avg       0.99      0.99      0.99      1989



In [20]:
zipped_lists = zip(predicted_labels, true_labels)

import pandas as pd
df = pd.DataFrame(zipped_lists, columns=['predicted_labels', 'true_labels'])
df.head(50)

,predicted_labels,true_labels
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [22]:
df.to_csv('Predictions/test_data_pred_vs_true.csv', index=False)  # Set index=False if you don't want to include row numbers
print("DataFrame has been written to file.")

DataFrame has been written to file.
